In [ ]:
import pandas as pd
import shutil
import os

''' 실 디렉토리로 변경하여 사용~~~  '''
ym  = '202401'
dir0  = f'd:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
file0 = f'{ym}_수입원장.xlsx'  #원본
# df = pd.read_excel(dir0+file0, sheet_name=['전체','메리츠화재'],dtype={'증권번호':'object'})
df = pd.read_excel(dir0+file0, sheet_name=None,dtype={'증권번호':object})

In [34]:
dict_s = {'AIG손해보험':'수금','DB손보':'수금관리','MG손보':'수금','흥국화재':'장기고객관리',
   '현대해상':'수금','한화손보':'장기고객관리','KB손보':'수금','롯데손보':'수금',
   '메리츠화재':'장기수금','농협손보':'수금','삼성화재':'장기수금',
   '하나손해보험':'수금수수료','처브손해보험':' '}
cols = ['지점','보험사','증권번호','상태','납입회차','보험료','수수료계']

In [37]:
df_all = pd.DataFrame()
qry = "계약종류 == '장기' and (지점 == '글래드피스토스' or 지점 == '글래드가온')"

for k,v in df.items():
    if k in dict_s.keys():
        temp_col = dict_s.get(k)
        if temp_col == ' ':
            df_temp = v.query(qry)[ cols].copy()
            df_temp['수금'] = 0
        else:
            df_temp = v.query(qry)[ cols + [temp_col] ].copy()
        df_temp.rename(columns={temp_col:'수금수수료'}, inplace=True)
        df_all = pd.concat([df_all, df_temp])


In [38]:
path_ibss = f'd:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
ibss_file = '마감실적리스트(장기)-202402270600.xlsx'
df_i = pd.read_excel(path_ibss+ibss_file, header=0, dtype={'증권번호':object})
qry = "지점 == '글래드피스토스' or 지점 == '글래드가온'"
df_i = df_i.query(qry)

In [39]:
qry = "지점 == '글래드피스토스' or 지점=='글래드가온'"
cols = ['지점','보험회사','증권번호','구분','월납보험료','수수료H']
df_i = df_i.query(qry)[cols]

In [40]:
df_all = pd.merge(df_all, df_i, how = 'left',
            on=['증권번호'],indicator = True).reset_index().astype({"_merge": str}).fillna(0)

In [43]:
df_all['차이'] = df_all['수수료계'] * 0.95 - df_all['수수료H']
df_all[df_all['차이'] !=0].to_clipboard()